In this notebook, the primary model is the one that sets a limit buy/sell order based on ATR and profit take/stop loss based on daily volatility.

In [1]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

bybit = ccxt.bybit({"apiKey":"", "secret":""})
bybit.set_sandbox_mode(True)

import math
from tqdm import tqdm


import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
import seaborn as sns
import talib

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score, KFold, TimeSeriesSplit

In [70]:
def get_daily_vol(close, lookback=100):
    
    """
    This function is credited to Marcos Lopez de Prado. 
    
    """
    print('Calculating daily volatility for dynamic thresholds')
    
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = (pd.Series(close.index[df0 - 1], index=close.index[close.shape[0] - df0.shape[0]:]))
        
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1  # daily returns
    df1 = df0.ewm(span=lookback).std()
    return df1

In [71]:
def getTEvents(gRaw, h):
    
    """
    The code is credited to Marcos Lopez de Prado.
    
    """
    tEvents, sPos, sNeg = [], 0, 0
    diff = np.log(gRaw).diff().dropna().abs()
    for i in diff.index[1:]:
        try:
            pos, neg = float(sPos+diff.loc[i]), float(sNeg+diff.loc[i])
        except Exception as e:
            print(e)
            print(sPos+diff.loc[i], type(sPos+diff.loc[i]))
            print(sNeg+diff.loc[i], type(sNeg+diff.loc[i]))
            break

        sPos, sNeg=max(0., pos), min(0., neg)
        if sNeg<-h:
            sNeg=0;tEvents.append(i)
        elif sPos>h:
            sPos=0;tEvents.append(i)
    return pd.DatetimeIndex(tEvents)


In [72]:
def get_dollar_bars(time_bars, dollar_threshold): 
    
    """
    The function is credit to Max Bodoia and partially revised by Yusaku Fujii
    
    """
    
    # Make ohlcv df a dictionary
    time_bars = time_bars.to_dict('records') 
    
    # initialize an empty list of dollar bars
    dollar_bars = []

    # initialize the running dollar volume at zero
    running_volume = 0

    # initialize the running high and low with placeholder values
    running_high, running_low = 0, math.inf

    # for each time bar...
    for i in range(len(time_bars)):

        # get the timestamp, open, high, low, close, and volume of the next bar
        next_close, next_high, next_low, next_open, next_timestamp, next_volume,next_fee = [time_bars[i][k] for k in ['close', 'high', 'low', 'open', 'timestamp', 'volume','maker fee']]

        # get the midpoint price of the next bar (the average of the open and the close)
        midpoint_price = ((next_open) + (next_close))/2

        # get the approximate dollar volume of the bar using the volume and the midpoint price
        dollar_volume = next_volume * midpoint_price

        # update the running high and low
        running_high, running_low = max(running_high, next_high), min(running_low, next_low)

        # if the next bar's dollar volume would take us over the threshold...
        if dollar_volume + running_volume >= dollar_threshold:

            # set the timestamp for the dollar bar as the timestamp at which the bar closed (i.e. one minute after the timestamp of the last minutely bar included in the dollar bar)
            bar_timestamp = next_timestamp + timedelta(minutes=5)
            
            # add a new dollar bar to the list of dollar bars with the timestamp, running high/low, and next close
            dollar_bars += [{'timestamp': bar_timestamp, 'open': next_open, 'high': running_high, 'low': running_low, 'close': next_close,'maker fee':next_fee}]

            # reset the running volume to zero
            running_volume = 0

            # reset the running high and low to placeholder values
            running_high, running_low = 0, math.inf

        # otherwise, increment the running volume
        else:
            running_volume += dollar_volume

    # change the list of dollar bars to its dataframe
    dollar_bars = pd.DataFrame(dollar_bars)
    
    # set timestamp as index
    dollar_bars.set_index('timestamp',inplace=True)
    
    # remove the date that duplicates
    dollar_bars=dollar_bars.drop_duplicates()
    
    return dollar_bars


In [73]:
features = ['ADX', 'APO', 'AROONOSC', 'AROON_aroondown', 'AROON_aroonup', 'ATR',
       'BBANDS_lowerband', 'BBANDS_middleband', 'BBANDS_upperband', 'BETA',
       'CCI', 'DX', 'EMA', 'HT_TRENDMODE', 'KAMA', 'LINEARREG',
       'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'MA',
       'MIDPOINT', 'MOM', 'RSI', 'STDDEV', 'STOCHF_fastk',
       'STOCH_slowd', 'STOCH_slowk', 'T3', 'TRIMA', 'ULTOSC', 'WILLR', 'WMA']

def calc_features(df):
    
    """
    This function is credited to richmanbtc.
    
    """
    open = df['open']
    high = df['high']
    low = df['low']
    close = df['close']
    #volume = df['volume']

    hilo = (df['high'] + df['low']) / 2
    df['BBANDS_upperband'], df['BBANDS_middleband'], df['BBANDS_lowerband'] = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    df['BBANDS_upperband'] -= hilo
    df['BBANDS_middleband'] -= hilo
    df['BBANDS_lowerband'] -= hilo
    df['DEMA'] = talib.DEMA(close, timeperiod=30) - hilo
    df['EMA'] = talib.EMA(close, timeperiod=30) - hilo
    df['HT_TRENDLINE'] = talib.HT_TRENDLINE(close) - hilo
    df['KAMA'] = talib.KAMA(close, timeperiod=30) - hilo
    df['MA'] = talib.MA(close, timeperiod=30, matype=0) - hilo
    df['MIDPOINT'] = talib.MIDPOINT(close, timeperiod=14) - hilo
    df['SMA'] = talib.SMA(close, timeperiod=30) - hilo
    df['T3'] = talib.T3(close, timeperiod=5, vfactor=0) - hilo
    df['TEMA'] = talib.TEMA(close, timeperiod=30) - hilo
    df['TRIMA'] = talib.TRIMA(close, timeperiod=30) - hilo
    df['WMA'] = talib.WMA(close, timeperiod=30) - hilo

    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['ADXR'] = talib.ADXR(high, low, close, timeperiod=14)
    df['APO'] = talib.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df['AROON_aroondown'], df['AROON_aroonup'] = talib.AROON(high, low, timeperiod=14)
    df['AROONOSC'] = talib.AROONOSC(high, low, timeperiod=14)
    df['BOP'] = talib.BOP(open, high, low, close)
    df['CCI'] = talib.CCI(high, low, close, timeperiod=14)
    df['DX'] = talib.DX(high, low, close, timeperiod=14)
    df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    # skip MACDEXT MACDFIX たぶん同じなので
    #df['MFI'] = talib.MFI(high, low, close, volume, timeperiod=14)
    df['MINUS_DI'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    df['MINUS_DM'] = talib.MINUS_DM(high, low, timeperiod=14)
    df['MOM'] = talib.MOM(close, timeperiod=10)
    df['PLUS_DI'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['PLUS_DM'] = talib.PLUS_DM(high, low, timeperiod=14)
    df['RSI'] = talib.RSI(close, timeperiod=14)
    df['STOCH_slowk'], df['STOCH_slowd'] = talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df['STOCHF_fastk'], df['STOCHF_fastd'] = talib.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = talib.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['TRIX'] = talib.TRIX(close, timeperiod=30)
    df['ULTOSC'] = talib.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    df['WILLR'] = talib.WILLR(high, low, close, timeperiod=14)

    #df['AD'] = talib.AD(high, low, close, volume)
    #df['ADOSC'] = talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
    #df['OBV'] = talib.OBV(close, volume)

    df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
    df['NATR'] = talib.NATR(high, low, close, timeperiod=14)
    df['TRANGE'] = talib.TRANGE(high, low, close)

    df['HT_DCPERIOD'] = talib.HT_DCPERIOD(close)
    df['HT_DCPHASE'] = talib.HT_DCPHASE(close)
    df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(close)
    df['HT_SINE_sine'], df['HT_SINE_leadsine'] = talib.HT_SINE(close)
    df['HT_TRENDMODE'] = talib.HT_TRENDMODE(close)

    df['BETA'] = talib.BETA(high, low, timeperiod=5)
    df['CORREL'] = talib.CORREL(high, low, timeperiod=30)
    df['LINEARREG'] = talib.LINEARREG(close, timeperiod=14) - close
    df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(close, timeperiod=14)
    df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(close, timeperiod=14) - close
    df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(close, timeperiod=14)
    df['STDDEV'] = talib.STDDEV(close, timeperiod=5, nbdev=1)

    return df

In [87]:
def compute_primary_model_ret_buy(df,cond,high,low,sl,tp,fee):
    
    """
    tp:Upper
    sl:Lower
    
    """
    exit_price=sl.copy()
    exit_price[:]=np.nan
    
    

    for i in range(exit_price.size):
        for j in range(i+1,exit_price.size):
            
            if cond[i]==1:
                if low[j]<sl[j]:
                    exit_price[i]=sl[j]
                    break
                    
                    
                elif high[j]>tp[j]:
                    exit_price[i]=tp[j]
                    break
                                  
    df['exit_price_buy']=exit_price
    df['y_buy']=df['exit_price_buy']/df['buy_price']-1-2*fee
    df.loc[df['y_buy'].isna(),'y_buy']=0
    
    return df

In [88]:
def compute_primary_model_ret_sell(df,cond,high,low,sl,tp,fee):
    
    """
    tp:Lower
    sl:Upper
    
    ・現状、slが優先的に処理されている。tpを経由してからslに到達するprice pathを、
    　＋リターンとして評価していない点が、優れている。
    
    """
    
    exit_price=sl.copy()
    exit_price[:]=np.nan

    for i in range(exit_price.size):
        for j in range(i+1,exit_price.size):
            
            if cond[i]==1:
                if high[j]>sl[j]:
                    exit_price[i]=sl[j]
                    break
                    
                elif low[j]<tp[j]:
                    exit_price[i]=tp[j]
                    break
                    
            
                    

    df['exit_price_sell']=exit_price
    df['y_sell']=df['exit_price_sell']/df['sell_price']-1-2*fee
    df.loc[df['y_sell'].isna(),'y_sell']=0
    
    return df

In [67]:
df=pd.read_parquet('btcusdt_2102_raw')
df

,open,high,low,close,volume,maker fee
timestamp,,,,,,
2021-02-21 00:45:00,56123.0,56231.0,55868.0,55967.5,98.816,-0.00025
2021-02-21 01:00:00,55967.5,56266.0,55883.5,56238.5,96.312,-0.00025
2021-02-21 01:15:00,56238.5,56252.5,56023.5,56054.0,88.322,-0.00025
2021-02-21 01:30:00,56054.0,56080.0,55911.5,55997.5,91.413,-0.00025
2021-02-21 01:45:00,55997.5,56356.0,55985.5,56356.0,79.990,-0.00025
...,...,...,...,...,...,...
2022-06-16 01:30:00,22687.0,22706.0,22515.0,22542.0,190.518,0.00010
2022-06-16 01:45:00,22542.0,22600.0,22361.5,22372.5,101.782,0.00010
2022-06-16 02:00:00,22372.5,22600.0,22360.0,22505.5,45.235,0.00010


In [76]:
df=get_dollar_bars(time_bars=df.reset_index("timestamp"), dollar_threshold=df['close'].mean()*df['volume'].mean())
df['daily vol']=get_daily_vol(close=df['close'].drop_duplicates(), lookback=100)

Calculating daily volatility for dynamic thresholds


In [77]:
df=df.dropna()
df

,open,high,low,close,maker fee,daily vol
timestamp,,,,,,
2021-02-22 01:35:00,56770.0,57127.0,56658.5,57015.5,-0.00025,0.003399
2021-02-22 01:50:00,57015.5,57062.0,56912.0,56985.0,-0.00025,0.002911
2021-02-22 02:05:00,56985.0,56985.0,56596.0,56780.5,-0.00025,0.004775
2021-02-22 02:20:00,56780.5,57141.5,56712.0,57071.5,-0.00025,0.004136
2021-02-22 02:35:00,57071.5,57313.5,57071.5,57159.0,-0.00025,0.004001
...,...,...,...,...,...,...
2022-06-16 00:20:00,22650.5,22694.0,22398.0,22555.0,0.00010,0.053747
2022-06-16 00:35:00,22555.0,22877.5,22464.5,22527.5,0.00010,0.054057
2022-06-16 01:05:00,22667.5,23032.5,22424.5,22951.5,0.00010,0.054733


In [78]:
df=df.drop(index=df.loc[df.index.duplicated()].index)

In [79]:
df.index.duplicated().sum()

0

In [81]:
df=calc_features(df)
df=df.dropna()

/tmp/ipykernel_721/3003935834.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BBANDS_upperband'], df['BBANDS_middleband'], df['BBANDS_lowerband'] = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
/tmp/ipykernel_721/3003935834.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BBANDS_upperband'] -= hilo
/tmp/ipykernel_721/3003935834.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [82]:
limit_price_dist = df['ATR']*0.5
pips=0.5
limit_price_dist = np.maximum(1,(limit_price_dist / pips).round().fillna(1)) * pips

In [83]:
df.loc[:,'buy_price']=df.loc[:,'close']-limit_price_dist
df.loc[:,'sell_price']=df.loc[:,'close']+limit_price_dist

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [84]:
upper_bound=((df.loc[df.index[1]:,'close']*0.2*df.loc[df.index[1]:,'daily vol'])/pips).round()*pips
lower_bound=((df.loc[df.index[1]:,'close']*0.4*df.loc[df.index[1]:,'daily vol'])/pips).round()*pips*-1

In [85]:
df['Upper']=df['close']+upper_bound
df['Lower']=df['close']+lower_bound

/tmp/ipykernel_721/963696436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Upper']=df['close']+upper_bound
/tmp/ipykernel_721/963696436.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lower']=df['close']+lower_bound


In [86]:
df['buy_executed']=np.where(df['buy_price']>df['low'].shift(-1),1,0)
df['sell_executed']=np.where(df['sell_price']<df['high'].shift(-1),1,0)

/tmp/ipykernel_721/835364444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['buy_executed']=np.where(df['buy_price']>df['low'].shift(-1),1,0)
/tmp/ipykernel_721/835364444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sell_executed']=np.where(df['sell_price']<df['high'].shift(-1),1,0)


In [89]:
df=compute_primary_model_ret_buy(df=df,cond=df['buy_executed'],high=df['high'],low=df['low'],sl=df['Lower'],tp=df['Upper'],fee=df['maker fee'])
df

/tmp/ipykernel_721/843148594.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['exit_price_buy']=exit_price
/tmp/ipykernel_721/843148594.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y_buy']=df['exit_price_buy']/df['buy_price']-1-2*fee
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,open,high,low,close,maker fee,daily vol,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,DEMA,...,LINEARREG_SLOPE,STDDEV,buy_price,sell_price,Upper,Lower,buy_executed,sell_executed,exit_price_buy,y_buy
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-02-23 05:50:00,49963.0,50600.0,48779.5,50060.0,-0.00025,0.029728,1565.719965,416.25,-733.219965,1207.837059,...,-218.503297,574.734982,49592.5,50527.5,NaN,NaN,1,1,50094.5,0.010622
2021-02-23 06:20:00,50842.0,51216.5,49530.0,50700.0,-0.00025,0.029862,809.986002,-288.85,-1387.686002,425.403967,...,-186.913187,549.418001,50206.0,51194.0,51003.0,50094.5,1,0,50472.5,0.005808
2021-02-23 06:35:00,50700.0,50863.5,50005.0,50169.0,-0.00025,0.030253,469.817542,-457.05,-1383.917542,211.108081,...,-183.987912,463.433771,49679.5,50658.5,50472.5,49562.0,1,0,50166.5,0.010303
2021-02-23 07:05:00,49740.0,50465.0,49474.0,49860.5,-0.00025,0.030670,911.688266,69.70,-772.288266,501.199109,...,-206.846154,420.994133,49370.5,50350.5,50166.5,49249.0,0,1,NaN,0.000000
2021-02-23 07:35:00,50062.5,50560.5,49606.5,50558.0,-0.00025,0.030639,811.992971,186.00,-439.992971,318.906714,...,-171.203297,312.996486,50069.0,51047.0,50868.0,49938.5,1,0,50109.5,0.001309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-16 00:20:00,22650.5,22694.0,22398.0,22555.0,0.00010,0.053747,145.796931,39.90,-65.996931,-189.536198,...,77.763736,52.948466,22305.5,22804.5,22797.5,22070.0,0,1,NaN,0.000000
2022-06-16 00:35:00,22555.0,22877.5,22464.5,22527.5,0.00010,0.054057,10.670640,-101.20,-213.070640,-266.317196,...,69.841758,55.935320,22281.0,22774.0,22771.0,22040.5,0,1,NaN,0.000000
2022-06-16 01:05:00,22667.5,23032.5,22424.5,22951.5,0.00010,0.054733,239.011717,-91.70,-422.411717,-228.163243,...,64.082418,165.355859,22701.0,23202.0,23202.5,22449.0,1,0,22936.5,0.010174


In [90]:
df=compute_primary_model_ret_sell(df=df,cond=df['sell_executed'],high=df['high'],low=df['low'],sl=df['Upper'],tp=df['Lower'],fee=df['maker fee'])
df

/tmp/ipykernel_721/881046222.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['exit_price_sell']=exit_price
/tmp/ipykernel_721/881046222.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y_sell']=df['exit_price_sell']/df['sell_price']-1-2*fee
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,open,high,low,close,maker fee,daily vol,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,DEMA,...,buy_price,sell_price,Upper,Lower,buy_executed,sell_executed,exit_price_buy,y_buy,exit_price_sell,y_sell
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-02-23 05:50:00,49963.0,50600.0,48779.5,50060.0,-0.00025,0.029728,1565.719965,416.25,-733.219965,1207.837059,...,49592.5,50527.5,NaN,NaN,1,1,50094.5,0.010622,51003.0,0.009911
2021-02-23 06:20:00,50842.0,51216.5,49530.0,50700.0,-0.00025,0.029862,809.986002,-288.85,-1387.686002,425.403967,...,50206.0,51194.0,51003.0,50094.5,1,0,50472.5,0.005808,NaN,0.000000
2021-02-23 06:35:00,50700.0,50863.5,50005.0,50169.0,-0.00025,0.030253,469.817542,-457.05,-1383.917542,211.108081,...,49679.5,50658.5,50472.5,49562.0,1,0,50166.5,0.010303,NaN,0.000000
2021-02-23 07:05:00,49740.0,50465.0,49474.0,49860.5,-0.00025,0.030670,911.688266,69.70,-772.288266,501.199109,...,49370.5,50350.5,50166.5,49249.0,0,1,NaN,0.000000,49938.5,-0.007683
2021-02-23 07:35:00,50062.5,50560.5,49606.5,50558.0,-0.00025,0.030639,811.992971,186.00,-439.992971,318.906714,...,50069.0,51047.0,50868.0,49938.5,1,0,50109.5,0.001309,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-16 00:20:00,22650.5,22694.0,22398.0,22555.0,0.00010,0.053747,145.796931,39.90,-65.996931,-189.536198,...,22305.5,22804.5,22797.5,22070.0,0,1,NaN,0.000000,22771.0,-0.001669
2022-06-16 00:35:00,22555.0,22877.5,22464.5,22527.5,0.00010,0.054057,10.670640,-101.20,-213.070640,-266.317196,...,22281.0,22774.0,22771.0,22040.5,0,1,NaN,0.000000,22449.0,-0.014471
2022-06-16 01:05:00,22667.5,23032.5,22424.5,22951.5,0.00010,0.054733,239.011717,-91.70,-422.411717,-228.163243,...,22701.0,23202.0,23202.5,22449.0,1,0,22936.5,0.010174,NaN,0.000000
